In [2]:
import pandas as pd     

In [3]:
#reading in CSV file
care_df = pd.read_csv("care.csv")
care_df.head()

,_id,Indicator,AACC Region,Aboriginality,Age Group,Gender,Count,Year
0,1,Children in care,East Kimberley,NonAboriginal,0 - 17 Years,All,0,2008
1,2,Children in care,East Kimberley,Aboriginal,0 - 17 Years,All,91,2008
2,3,Children in care,Goldfields-Esperance,NonAboriginal,0 - 17 Years,All,34,2008
3,4,Children in care,Goldfields-Esperance,Aboriginal,0 - 17 Years,All,69,2008
4,5,Children in care,Midwest-Gascoyne,Aboriginal,0 - 17 Years,All,94,2008


In [4]:
#reading in CSV file
assessed_df = pd.read_csv("assessed.csv")
assessed_df.head()

,_id,Indicator,AACC Region,Aboriginality,Age Group,Gender,Count,Year
0,1,Safety and Wellbeing Assessments,East Kimberley,Aboriginal,0 - 17 Years,All,325,2015-16
1,2,Safety and Wellbeing Assessments,East Kimberley,NonAboriginal,0 - 17 Years,All,4,2015-16
2,3,Safety and Wellbeing Assessments,Goldfields-Esperance,Aboriginal,0 - 17 Years,All,193,2015-16
3,4,Safety and Wellbeing Assessments,Goldfields-Esperance,NonAboriginal,0 - 17 Years,All,126,2015-16
4,5,Safety and Wellbeing Assessments,Midwest-Gascoyne,Aboriginal,0 - 17 Years,All,471,2015-16


In [5]:
# Delete age group column (not required)
del care_df['Age Group']
care_df.head()

,_id,Indicator,AACC Region,Aboriginality,Gender,Count,Year
0,1,Children in care,East Kimberley,NonAboriginal,All,0,2008
1,2,Children in care,East Kimberley,Aboriginal,All,91,2008
2,3,Children in care,Goldfields-Esperance,NonAboriginal,All,34,2008
3,4,Children in care,Goldfields-Esperance,Aboriginal,All,69,2008
4,5,Children in care,Midwest-Gascoyne,Aboriginal,All,94,2008


In [7]:
#care2_df = care_df.set_index('Year')

In [8]:
#care2_df

,_id,Indicator,AACC Region,Aboriginality,Gender,Count
Year,,,,,,
2008,1,Children in care,East Kimberley,NonAboriginal,All,0
2008,2,Children in care,East Kimberley,Aboriginal,All,91
2008,3,Children in care,Goldfields-Esperance,NonAboriginal,All,34
2008,4,Children in care,Goldfields-Esperance,Aboriginal,All,69
2008,5,Children in care,Midwest-Gascoyne,Aboriginal,All,94
...,...,...,...,...,...,...
2017,176,Children in care,Western Australia,NonAboriginal,All,2192
2017,177,Children in care,Wheatbelt,Aboriginal,All,163
2017,178,Children in care,Wheatbelt,NonAboriginal,All,90


In [10]:
#delete years prior to 2015
care2_df = care_df.loc[care_df['Year']>2014]
care2_df.head()

,_id,Indicator,AACC Region,Aboriginality,Gender,Count,Year
126,127,Children in care,East Kimberley,Aboriginal,All,144,2015
127,128,Children in care,East Kimberley,NonAboriginal,All,0,2015
128,129,Children in care,Goldfields-Esperance,NonAboriginal,All,23,2015
129,130,Children in care,Goldfields-Esperance,Aboriginal,All,135,2015
130,131,Children in care,Midwest-Gascoyne,Aboriginal,All,142,2015


In [11]:
#Delete years after 2016
care3_df = care2_df.loc[care2_df['Year']<2017]
care3_df.tail()

,_id,Indicator,AACC Region,Aboriginality,Gender,Count,Year
157,158,Children in care,Western Australia,NonAboriginal,All,2178,2016
158,159,Children in care,Wheatbelt,Aboriginal,All,142,2016
159,160,Children in care,Wheatbelt,NonAboriginal,All,101,2016
160,161,Children in care,West Kimberley,Aboriginal,All,189,2016
161,162,Children in care,West Kimberley,NonAboriginal,All,1,2016
